# <center>Big Data &ndash; Exercises</center>
## <center>Fall 2021 &ndash; Week 9 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and Spark SQL, Moodle exercise</center>

# Preparation for the moodle exercise in Spark

In this jupyter notebook we are going to make the preprocessing part of the dataset that is going to be used in the graded exercise of this week.
It will be the same language game dataset as in exercise08.

1. Change to exercise09 repository

2. Start docker <br>
```docker-compose up -d```

3. Getting the data:
Follow the procedure that is described below. The dataset can be found here: http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2. 

More specifically do the following:
- download the data      :<br> ```wget http://data.greatlanguagegame.com.s3.amazonaws.com/confusion-2014-03-02.tbz2```
- extract the data       :<br> ```tar -jxvf confusion-2014-03-02.tbz2```

4. copy the data to docker :<br> ```docker cp confusion-2014-03-02/confusion-2014-03-02.json jupyter:/home/jovyan/work``` <br>
(Copying the data to docker needs to be done only once and it might take 1-2 minutes.)

## More Info about the data
You can find more information about the dataset (as well as the schema and examples) in this link: http://lars.yencken.org/datasets/languagegame/

## Instructions:

In every query we ask you for three quantities: the query itself, the result of the query as well as the productivity time. That means the development time of each query (time elapsed before you start writing the query, and the time at which the correct, final query is ready). Note that the time part of every question is optional and not graded. In order to make easier the time recording we created two functions that do it automatically. Run the cell below in order to import the functions into the current notebook. Then before each query we will have a ```start_exercise()``` cell that you have to run in order to start time recording. After you have finished your query and you are sure about the answer run the ```finish_exercise()``` one to get the time measurement. 

In [1]:
import time

def start_exercise():
    global last
    last = time.time()
    
def finish_exercise():
    global last
    print("This exercise took {0}s".format(int(time.time()-last)))

## <center>1. Spark Dataframes</center>

Write queries for the same questions as last week, but this time using Spark Dataframes operations (the data loading will take a minute)

### 1.0. Data preprocessing

In [2]:
import json
from pyspark.sql import SparkSession
from pyspark import SparkConf

spark = SparkSession.builder.master('local').getOrCreate()
sc = spark.sparkContext

path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()

In [3]:
dataset.printSchema()

root
 |-- choices: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- country: string (nullable = true)
 |-- date: string (nullable = true)
 |-- guess: string (nullable = true)
 |-- sample: string (nullable = true)
 |-- target: string (nullable = true)



## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [4]:
start_exercise()

In [5]:
dataset.filter((dataset["guess"] == dataset["target"]) & (dataset["guess"] == "Russian")).count()

290818

In [6]:
finish_exercise()

This exercise took 55s


## Assignment 2
Return the number of distinct "target" languages.

In [7]:
start_exercise()

In [8]:
dataset.dropDuplicates(["target"]).select(dataset["target"]).count()

78

In [9]:
finish_exercise()

This exercise took 4s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [10]:
start_exercise()

In [11]:
dataset.filter(dataset["guess"] == dataset["target"]).orderBy(["target", "country", "date"], ascending=True).select("sample").limit(3).show(20, False)

+--------------------------------+
|sample                          |
+--------------------------------+
|00b85faa8b878a14f8781be334deb137|
|efcd813daec1c836d9f030b30caa07ce|
|efcd813daec1c836d9f030b30caa07ce|
+--------------------------------+



In [12]:
finish_exercise()

This exercise took 7s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [13]:
start_exercise()

In [14]:
from pyspark.sql.functions import desc

dataset.groupBy(dataset["country"], dataset["target"]).agg({'guess':'count'}).orderBy(desc("count(guess)")).select("count(guess)").limit(3).show()

+------------+
|count(guess)|
+------------+
|      112934|
|      112007|
|      110919|
+------------+



In [15]:
finish_exercise()

This exercise took 6s


## Assignment 5
Find the percentage of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [16]:
start_exercise()

In [17]:
round(dataset.filter((dataset["guess"]==dataset["target"]) & (dataset["guess"]==dataset["choices"].getItem(0))).count() / dataset.count(),4)

0.256

In [18]:
finish_exercise()

This exercise took 5s


## Assignment 6
Sort the languages by decreasing overall percentage of correct guesses and return the first three languages.

In [70]:
start_exercise()

In [80]:
from pyspark.sql.functions import count, col

dataset.filter(dataset['target']==dataset['guess']).groupBy(dataset['target']).agg(count('target').alias('cnt_correct'))\
.join(dataset.groupBy(dataset['target']).agg(count('target').alias('cnt_overall')),on='target',how='inner').orderBy(desc(col('cnt_correct')/col('cnt_overall'))).select('target').limit(3).show()

+-------+
| target|
+-------+
| French|
| German|
|Spanish|
+-------+



In [79]:
finish_exercise()

This exercise took 637s


## Assignment 7
Return the number of games played on the latest day.

In [21]:
start_exercise()

In [22]:
dataset.groupBy(dataset['date']).agg({'date':'count'}).orderBy(desc(dataset['date'])).select('count(date)').limit(1).show()

+-----------+
|count(date)|
+-----------+
|      65653|
+-----------+



In [23]:
finish_exercise()

This exercise took 3s


## <center>2. Spark SQL</center>

Write Spark SQL queries for the same questions as earlier.

### 2.0. Data preprocessing

In [ ]:
!pip install sparksql-magic

In [25]:
%load_ext sparksql_magic

In [26]:
path = "confusion-2014-03-02.json"
dataset = spark.read.json(path).cache()
dataset.registerTempTable("dataset")

## Assignment 1
Find the number of games where the guessed language is correct (meaning equal to the target one) and that language is Russian.

In [28]:
start_exercise()

In [29]:
%%sparksql

SELECT COUNT(*) FROM dataset WHERE guess=target AND target="Russian"

count(1)
290818


In [30]:
finish_exercise()

This exercise took 2s


## Assignment 2
Return the number of distinct "target" languages.

In [31]:
start_exercise()

In [32]:
%%sparksql

SELECT COUNT(*) FROM (SELECT DISTINCT target FROM dataset)

count(1)
78


In [33]:
finish_exercise()

This exercise took 3s


## Assignment 3
Return the sample IDs (i.e., the *sample* field) of the top three games where the guessed language is correct (equal to the target one) ordered by language (ascending), then by country (ascending), then by date (ascending).

In [34]:
start_exercise()

In [35]:
%%sparksql

SELECT sample FROM dataset WHERE guess = target ORDER BY target ASC, country ASC, date ASC LIMIT 3

sample
00b85faa8b878a14f8781be334deb137
efcd813daec1c836d9f030b30caa07ce
efcd813daec1c836d9f030b30caa07ce


In [36]:
finish_exercise()

This exercise took 6s


## Assignment 4
Aggregate all games by country and target language, counting the number of guesses for each group and return the frequencies of the three most frequent country/language combinations.

In [37]:
start_exercise()

In [38]:
%%sparksql

SELECT COUNT(guess) AS freq FROM dataset GROUP BY country, target ORDER BY freq DESC LIMIT 3

freq
112934
112007
110919


In [39]:
finish_exercise()

This exercise took 5s


## Assignment 5
Find the percentage of games where (the answer was correct && the correct guess was the first choice amongst the array of possible answers)

Please write the fraction rounding to 4 decimals (eg. 0.3323)

In [40]:
start_exercise()

In [41]:
%%sparksql

SELECT round((SELECT COUNT(*) FROM dataset WHERE target=guess AND guess=choices[0]) / COUNT(*) ,4) FROM dataset

"round((CAST(scalarsubquery() AS DOUBLE) / CAST(count(1) AS DOUBLE)), 4)"
0.256


In [42]:
finish_exercise()

This exercise took 4s


## Assignment 6
Sort the languages by decreasing overall percentage of correct guesses and return the first three languages.

In [48]:
start_exercise()

In [68]:
%%sparksql
SELECT overall.target
FROM (
    SELECT target, COUNT(*) AS cnt_correct
    FROM dataset
    WHERE target=guess
    GROUP BY target) AS correct 
    JOIN (
    SELECT target, COUNT(*) AS cnt_overall
    FROM dataset
    GROUP BY target) AS overall 
    ON correct.target = overall.target
ORDER BY cnt_correct/cnt_overall DESC
LIMIT 3

target
French
German
Spanish


In [69]:
finish_exercise()

This exercise took 936s


## Assignment 7
Return the number of games played on the latest day.

In [45]:
start_exercise()

In [46]:
%%sparksql

SELECT COUNT(*) FROM dataset GROUP BY date ORDER BY date DESC LIMIT 1

count(1)
65653


In [47]:
finish_exercise()

This exercise took 3s
